# Annexe : le modèle de départ, en utilisant les matrices

On cherche ici à réaliser le modèle le plus simple possible.

* On prend une population de n individus.
* Ces individus sont connectés les uns aux autres par des liens, en réseau, générés de façon aléatoire, selon diverses distributions.
* Parmi ces n individus, un pourcentage p est touché par une maladie.
* La probabilité d'être touché par cette malade est d'une probabilité $\lambda$
* Certains individus sont immunisés contre la maladie (par un vaccin), avec une probabilité $\mu$.
* La probabilité de guérir sans la vaccin est de $\eta$.
* On suppose que, quand on est vacciné, la probabilité de tomber malade est de 0.

Ces hypothèses seront par la suite assouplies

Le temps t sera défini par l'utilisateur

In [2]:
import networkx as nx
import math as math
import random as rd
import numpy as np
import scipy.stats as scstat
import matplotlib as plot
import matplotlib.pyplot as plt

In [108]:
# Création du graphe et demande des variables

G = nx.Graph()
n = int(input('Taille de la population : '))
p_init = float(input ('Pourcentage initial de personnes contaminées : '))
lmbda = float(input('Contagiosité de la maladie : '))
mu = float(input('Poucentage de personnes vaccinées initialement : ')) 
eta = float(input('Probabilité que la personne guérisse sans vaccin : '))
rho = float(input('Probabilité de deces suite a la maladie : '))

Taille de la population : 200
Pourcentage initial de personnes contaminées : 0.01
Contagiosité de la maladie : 0.5
Poucentage de personnes vaccinées initialement : 0.4
Probabilité que la personne guérisse sans vaccin : 0.5
Probabilité de deces suite a la maladie : 0.5


In [138]:
# Création de la liste des malades initiaux (1 si malade, 0 si sain)

Population = np.random.choice([0, 1], size=(n,), p=[1-p_init, p_init])

#Création de la liste des personnes vaccinées (1 tu es vacciné, 0 pas vacciné)  

Population2 = np.random.choice([0, 1], size=(n,), p=[1-mu, mu])
Population = np.concatenate((Population,Population2.T), axis = 0)
Population = np.reshape(Population, (2,n))

In [139]:
Tmalade = np.asarray(np.zeros((n,1))) #Création de la durée de temps malade pour une personne

for i in range(n):
    if Population[0][i] == 1:
        Tmalade[i] += 1
    elif Population[0][i] == 1:
        Tmalade[i] = 0
        
#NB : les personnes qui guérissent de la maladie sont considérées comme vaccinées 

In [140]:
# Création d'une base de données comportant le temps et la fréquence des gens malades dans la population

Frequence = []
Frequence.append((0,float(sum(Population[0])/n)))
print(Frequence)

[(0, 0.01)]


## Implémentation du réseau

Dans cette partie, nous mettons en place un réseau généré aléatoirement par une fonction gaussienne. Les paramètres pourront être modifiés par les utilisateurs du logiciel.


In [231]:
#Création du réseau d'individus

G.clear()

m = float(input ('Moyenne de la taille d un cluster : '))
p_intra = float(input ('Probabilité de créer des liens au sein des clusters : '))
p_inter = float(input('Probabilité de créer des liens entre les clusters : '))
v = float(input('Paramètre de taille (variance) : ')) 

G = nx.gaussian_random_partition_graph(n,m,v,p_intra,p_inter)

#Premier paramètre : nombre de noeuds
#Deuxième paramètre : la moyenne de la taille des clusters
#Troisième paramètre : paramètre de taille, la variance de la taille des clusters vaut : moyenne de la taille / ce paramètre
#Quatrième paramètre : La probabilité de créer des liens au sein des clusters
#Cinquième paramètre : La probabilité de créer des liens entre les clusters

Moyenne de la taille d un cluster : 50
Probabilité de créer des liens au sein des clusters : 0.5
Probabilité de créer des liens entre les clusters : 0.1
Paramètre de taille (variance) : 100000


In [143]:
M = to_numpy_matrix(G)
print(M)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


## Début du processus dynamique 

In [145]:
#Demande du temps d'etude

t = int(input('Temps t : '))


# Première étape : vérification de la commande

#Transmission de la maladie
for l in range(t):
    for i in range(len(M)):
        for j in range (i,len(M)):
            if np.asarray(M)[i][j] == 1:
                if Population[0][i] == 1 & Population[0][j] == 0:
                    if Population[1][j] == 0:
                        Population[0][j] = int(np.random.binomial(1,lmbda))
                if Population[0][i] == 0 & Population[0][j] == 1:
                    if Population[1][i] == 0:
                        Population[0][i] = int(np.random.binomial(1,lmbda))
#Possibilité d'une guérison sans vaccin
        if Population[0][i] == 1 & int(Tmalade[i]) != 0:
            Population[0][i] = int(np.random.binomial(1,1-eta))
            if Population[0][i] == 0:
                Population[1][i] = 1 
#Mise à jour des décès            
    death_row_node = []
    death_row_index = []
    v = 0
    for i in G.nodes():
        if int(Tmalade[v]) != 0:
            k = int(np.random.binomial(1,rho))
            if k == 1:
                death_row_node.append(i)
                death_row_index.append(v)
        v = v+1         
#Mise à jour de la base de données sur la convalescence.        
    for i in range(len(M)):
        if Population[0][i] == 1:
            Tmalade[i] += 1
        elif Population[0][i] == 0:
            Tmalade[i] = 0
#Mise à jour des matrices en tenant compte des décès          
    G.remove_nodes_from(death_row_node)
    death_row_index.reverse()
    for i in death_row_index:
        Population = np.delete(Population,(i), axis=1)
        Tmalade = np.delete(Tmalade,(i), axis=0) 
    M = to_numpy_matrix(G)

Temps t : 10
